<h1 style="color:#1398A1">TRAINING<h1>


<h3 style="color:#CCC229">LIBRAIRIES IMPORT<h3>


In [44]:
import sys
from pathlib import Path

# Ajouter le dossier src au path
src_path = Path("..") / ".."
sys.path.append(str(src_path))
%load_ext autoreload
%autoreload 2

from pathlib import Path
from datetime import datetime
import uuid

import mlflow
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from huggingface_hub import HfApi
from env.snake_env import SnakeEnv
from utils.display import display_training_summary
from stable_baselines3.common.callbacks import BaseCallback

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h3 style="color:#CCC229">TRAIN FUNCTION<h3>

In [48]:
import os
import uuid
import json
import mlflow
import tempfile
from datetime import datetime
from pathlib import Path
from huggingface_hub import HfApi
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.utils import safe_mean
from dotenv import load_dotenv
import textwrap

load_dotenv()
hf_token = os.getenv("HF_HUB_TOKEN")
if not hf_token:
    raise ValueError("⚠️ Variable HF_HUB_TOKEN manquante.")

# ------------------------------------------
# Callback
# ------------------------------------------
class MLflowLoggingCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
    def _on_step(self) -> bool:
        return True
    def _on_rollout_end(self) -> None:
        try:
            logger_values = getattr(self.logger, "name_to_value", {})
            metrics = {k: float(v) for k, v in logger_values.items()}
            if hasattr(self.model, "ep_info_buffer") and len(self.model.ep_info_buffer) > 0:
                metrics["rollout/ep_rew_mean"] = float(safe_mean([ep["r"] for ep in self.model.ep_info_buffer]))
                metrics["rollout/ep_len_mean"] = float(safe_mean([ep["l"] for ep in self.model.ep_info_buffer]))
            if metrics:
                mlflow.log_metrics(metrics, step=self.num_timesteps)
        except Exception:
            pass

# ------------------------------------------
# Fonction d'entraînement
# ------------------------------------------
def train_snake(
    timesteps: int = 100_000,
    grid_size: int = 5,
    n_envs: int = 4,
    algorithm: str = "PPO",
    hf_repo_id: str = "Lategardener/snake-rl-models"
):
    agent_uuid = str(uuid.uuid4())
    date_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = f"{date_str}_{agent_uuid}"

    mlflow.set_experiment(f"Snake_{grid_size}x{grid_size}")

    with mlflow.start_run(run_name=run_name):

        # Tags de base
        mlflow.set_tag("agent_uuid", agent_uuid)
        mlflow.set_tag("hf_repo", hf_repo_id)
        mlflow.log_params({
            "algorithm": algorithm,
            "grid_size": grid_size,
            "total_timesteps": timesteps
        })

        print(f"🚀 Training: {grid_size}x{grid_size} | UUID: {agent_uuid}")

        env = make_vec_env(lambda: Monitor(SnakeEnv(grid_size=grid_size, render_mode=None)), n_envs=n_envs)
        agent = PPO("MlpPolicy", env, verbose=1)
        agent.learn(total_timesteps=timesteps, callback=MLflowLoggingCallback())

        print("💾 Chargement du model et des métrics vers le cloud...")

        with tempfile.TemporaryDirectory() as temp_dir_str:
            temp_dir = Path(temp_dir_str)
            model_path = temp_dir / "model.zip"
            meta_path = temp_dir / "metadata.json"

            # Sauvegarde locale (temporaire)
            agent.save(model_path)

            # Préparation des URLs Hugging Face
            hf_folder = f"{grid_size}x{grid_size}/{agent_uuid}"

            # URL du dossier complet
            hf_folder_url = f"https://huggingface.co/{hf_repo_id}/tree/main/{hf_folder}"
            # URL directe pour télécharger le ZIP
            hf_zip_url = f"https://huggingface.co/{hf_repo_id}/resolve/main/{hf_folder}/model.zip?download=true"

            # Metadata
            final_reward = None
            if hasattr(agent, "ep_info_buffer") and agent.ep_info_buffer:
                 final_reward = safe_mean([ep["r"] for ep in agent.ep_info_buffer])

            metadata = {
                "uuid": agent_uuid,
                "algorithm": algorithm,
                "grid_size": grid_size,
                "timesteps": timesteps,
                "date": date_str,
                "final_mean_reward": final_reward,
                "hf_folder_url": hf_folder_url,
                "hf_zip_url": hf_zip_url
            }

            with open(meta_path, "w") as f:
                json.dump(metadata, f, indent=4)

            # Upload vers HF
            api = HfApi(token=hf_token)
            api.create_repo(repo_id=hf_repo_id, repo_type="model", exist_ok=True, private=True)
            api.upload_folder(
                folder_path=str(temp_dir),
                path_in_repo=hf_folder,
                repo_id=hf_repo_id,
                commit_message=f"Add model {agent_uuid}"
            )

            # --- 4. Intégration Visuelle MLflow (La Magie est ici) ---
            # On utilise le tag spécial 'mlflow.note.content' qui affiche du Markdown
            # sur la page d'accueil du Run.

            readme_content = textwrap.dedent(f"""\
                # Modele Snake {grid_size}x{grid_size}

                ID Unique : {agent_uuid}
                Date : {date_str}
                Algorithme : {algorithm}
                Reward Finale (moy) : {final_reward if final_reward else 'N/A'}

                ---------------------------------------------------
                ACCES AUX FICHIERS (Hugging Face Hub)
                ---------------------------------------------------

                - [Telecharger le Modele (.zip)]({hf_zip_url})
                - [Voir le dossier complet sur HF]({hf_folder_url})
                """)

            mlflow.set_tag("mlflow.note.content", readme_content)

            # On ajoute aussi l'URL comme un tag simple pour pouvoir cliquer dessus dans la liste
            mlflow.set_tag("model_url", hf_folder_url)

            print("Entraînement terminé.")

In [49]:
train_snake(timesteps=200000, grid_size=10, n_envs=8, algorithm="PPO")

🚀 Training: 10x10 | UUID: 89015953-1afd-4db8-85dd-b4b2945e1952
Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30.3     |
|    ep_rew_mean     | -1.41    |
| time/              |          |
|    fps             | 8593     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 29.1        |
|    ep_rew_mean          | -1.43       |
| time/                   |             |
|    fps                  | 3128        |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.012470949 |
|    clip_fraction        | 0.136       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38 

Processing Files (0 / 0): |          |  0.00B /  0.00B            
Processing Files (1 / 1): 100%|██████████|  293kB /  293kB,  366kB/s  
Processing Files (1 / 1): 100%|██████████|  293kB /  293kB,  293kB/s  
New Data Upload: 100%|██████████|  293kB /  293kB,  293kB/s  


Entraînement terminé.
